# Capstone Part 3 Preliminary Modelling and Model Selection

This file contains some preliminary modelling done with the data that has been created from the previous step. We explore Logistic Regression, KNN, Decision Trees and Random Forests. The purpose of this exploration is to select a model to proceed with for building my App which will use the model prediction proba scores as the number to evaluate a Nasus Players performance that game. I also want this model to have non-linear boundaries as it is a more flexible model.

First import standard libraries.

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [2]:
# Set pandas to show max columns and max rows
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
# Read in clean data

match_data_df = pd.read_csv('MatchDataClean_complete.csv')

print(match_data_df.shape)
match_data_df.head()

(11818, 279)


,stats.win,stats.kills,stats.deaths,stats.assists,stats.largestKillingSpree,stats.largestMultiKill,stats.killingSprees,stats.longestTimeSpentLiving,stats.doubleKills,stats.tripleKills,stats.quadraKills,stats.pentaKills,stats.magicDamageDealt,stats.physicalDamageDealt,stats.trueDamageDealt,stats.largestCriticalStrike,stats.totalHeal,stats.totalUnitsHealed,stats.damageSelfMitigated,stats.damageDealtToObjectives,stats.visionScore,stats.timeCCingOthers,stats.totalDamageTaken,stats.goldEarned,stats.turretKills,stats.inhibitorKills,stats.totalMinionsKilled,stats.neutralMinionsKilled,stats.neutralMinionsKilledTeamJungle,stats.neutralMinionsKilledEnemyJungle,stats.totalTimeCrowdControlDealt,stats.visionWardsBoughtInGame,stats.wardsPlaced,stats.wardsKilled,stats.firstBloodKill,stats.firstBloodAssist,stats.firstTowerKill,stats.firstTowerAssist,stats.firstInhibitorKill,stats.firstInhibitorAssist,gameDuration,Cleanse,Exhaust,Flash,Ghost,Heal,Ignite,Smite,Teleport,'Your Cut',Abyssal Mask,Adaptive Helm,Aegis of the Legion,Aether Wisp,Amplifying Tome,Archangel's Staff,Ardent Censer,B. F. Sword,Bami's Cinder,Banshee's Veil,Berserker's Greaves,Bilgewater Cutlass,Black Cleaver,Blade of the Ruined King,Blasting Wand,Bloodthirster,Boots of Mobility,Boots of Speed,Boots of Swiftness,Bramble Vest,Broken Stopwatch,Bulwark of the Mountain,Catalyst of Aeons,Caulfield's Warhammer,Chain Vest,Cloak of Agility,Cloth Armor,Commencing Stopwatch,Control Ward,Corrupting Potion,Crystalline Bracer,Cull,Dagger,Dark Seal,Dead Man's Plate,Death's Dance,Doran's Blade,Doran's Ring,Doran's Shield,Duskblade of Draktharr,Edge of Night,Elixir of Iron,Elixir of Wrath,Enchantment: Bloodrazor,Enchantment: Cinderhulk,Enchantment: Runic Echoes,Enchantment: Warrior,Essence Reaver,Executioner's Calling,Eye of the Herald,Faerie Charm,Farsight Alteration,Fiendish Codex,Forbidden Idol,Frostfang,Frozen Fist,Frozen Heart,Frozen Mallet,Gargoyle Stoneplate,Giant's Belt,Glacial Shroud,Guardian Angel,Guinsoo's Rageblade,Haunting Guise,Health Potion,Hexdrinker,Hextech GLP-800,Hextech Gunblade,Hextech Protobelt-01,Hextech Revolver,Hunter's Machete,Hunter's Talisman,Iceborn Gauntlet,Infinity Edge,Ionian Boots of Lucidity,Jaurim's Fist,Kindlegem,Kircheis Shard,Knight's Vow,Last Whisper,Liandry's Torment,Lich Bane,Locket of the Iron Solari,Long Sword,Lord Dominik's Regards,Lost Chapter,Luden's Echo,Luden's Pulse,Manamune,Maw of Malmortius,Mejai's Soulstealer,Mercurial Scimitar,Mercury's Treads,Minion Dematerializer,Morellonomicon,Mortal Reminder,Muramana,Nashor's Tooth,Needlessly Large Rod,Negatron Cloak,Ninja Tabi,Null-Magic Mantle,Oblivion Orb,Oracle Lens,Pauldrons of Whiterock,Perfectly Timed Stopwatch,Phage,Phantom Dancer,Pickaxe,Quicksilver Sash,Rabadon's Deathcap,Randuin's Omen,Rapid Firecannon,Ravenous Hydra,Recurve Bow,Redemption,Refillable Potion,Rejuvenation Bead,Righteous Glory,Rod of Ages,Ruby Crystal,Runesteel Spaulders,Rylai's Crystal Scepter,Sanguine Blade,Sapphire Crystal,Seeker's Armguard,Seraph's Embrace,Serrated Dirk,Shard of True Ice,Sheen,Shurelya's Reverie,Skirmisher's Sabre,Slightly Magical Boots,Sorcerer's Shoes,Spear of Shojin,Spectre's Cowl,Spellbinder,Spirit Visage,Stalker's Blade,Statikk Shiv,Sterak's Gage,Stinger,Stopwatch,Stormrazor,Sunfire Cape,Tear of the Goddess,Thornmail,Tiamat,Titanic Hydra,Total Biscuit of Everlasting Will,Trinity Force,Trinity Fusion,Twin Shadows,Vampiric Scepter,Void Staff,Warden's Mail,Warding Totem (Trinket),Warmog's Armor,Wit's End,Youmuu's Ghostblade,Zeal,Zeke's Convergence,Zhonya's Hourglass,Fraction Magic Damage Delt to Champions,Fraction Physical Damage Delt to Champions,Fraction True Damage Delt to Champions,Fraction of damage dealt to turrets,AbsoluteFocus,ApproachVelocity,ArcaneComet,BiscuitDelivery,BonePlating,Celerity,CheapShot,Conditioning,Conqueror,CosmicInsight,CoupDeGrace,CutDown,DarkHarvest,Demolish,Electrocute,EyeballCollection,FleetFootwork,FontOfLife,FuturesMarket,GatheringStorm,GhostPoro,GlacialAugment,GraspOfTheUnd

In [5]:
# Check value counts for wins
match_data_df['stats.win'].value_counts()

0    5938
1    5880
Name: stats.win, dtype: int64

### Logistic Regression Model

In [70]:
# Import
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [71]:
# Set target and features
y = match_data_df['stats.win']
X = match_data_df.drop('stats.win', axis=1)

In [72]:
# Do a train test split
split = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)

In [73]:
# Scale the Features using the standard scaler
# Instantiate
std_scaler = StandardScaler()

# Fit and Transform
X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.fit_transform(X_test)

In [85]:
# Instantiate with l1 penalty as to
log_reg = LogisticRegression(penalty='l1', solver='liblinear')

# fit to scaled training data
log_reg_model = log_reg.fit(X_train_scaled, y_train)

# score on scaled training data
print(f"The accuracy on the training set is: {log_reg_model.score(X_train_scaled,y_train)}")

# score on scaled test data 
print(f"The accuracy on the test set is: {log_reg_model.score(X_test_scaled,y_test)}")

The accuracy on the training set is: 0.8953881954728158
The accuracy on the test set is: 0.8942470389170897


This is a pretty decent accuracy score for the train and test sets of logistic regression model with a lasso penalty. The reason why I chose lasso penalty was to reduce some of the coeficients of the features to zero, thereby eliminating them from the model. Hopefully this will remove (if any) some multicolinearity.

In [80]:
# Create a dictionary for coefficient dataframe
Coefficients = {'Feature':  list(X.columns),
        'Coefficients': log_reg_model.coef_[0]}

In [83]:
# create the coefficient dataframe
coef_df = pd.DataFrame(Coefficients, columns = ['Feature','Coefficients'])

# check
coef_df

,Feature,Coefficients
0,stats.kills,-0.120366
1,stats.deaths,-0.965893
2,stats.assists,1.138709
3,stats.largestKillingSpree,-0.070636
4,stats.largestMultiKill,0.266160
5,stats.killingSprees,0.018946
6,stats.longestTimeSpentLiving,-0.565178
7,stats.doubleKills,-0.104468
8,stats.tripleKills,-0.077804
9,stats.quadraKills,0.095364


In [82]:
# Sort the Dataframe and show top 5 and lowest 5 factors affecting affecting match outcome for Nasus Players
display(coef_df.sort_values(['Coefficients'], ascending=False).head(5))
print('\n')
display(coef_df.sort_values(['Coefficients'], ascending=True).head(5))

,Feature,Coefficients
22,stats.goldEarned,3.311993
2,stats.assists,1.138709
18,stats.damageDealtToObjectives,1.035951
15,stats.totalHeal,0.770526
38,stats.firstInhibitorAssist,0.694144


,Feature,Coefficients
199,Trinity Force,-1.053925
1,stats.deaths,-0.965893
21,stats.totalDamageTaken,-0.878005
186,Spirit Visage,-0.800172
195,Thornmail,-0.677442


This model suggests that the top 5 features for successfull Nasus players are:

1. Gold Earned: Gold earned is a raw representation how well a player is doing in a match. The better the player is performing, the higher his gold. Most gold comes from destroying minions, objectives and other champions. Gold can also help player buy items that improve their chances of wining the game. This feature makes logical sense as being the highest factor in a Nasus player winning a game.

2. Assists: Assists are granted when another player eliminates a champion that the Nasus player was engaged with. This could suggest that Nasus players who win games are supporting there teams in more team fights.

3. Damage Dealt to Objectives: This represents the damage dealt to all Objectives in the game (Towers, Inhibitors, Nexus and Neutral Jungle Camps). This would suggest that Nasus player who deal more damage to these objectives more likelt to win. This makes sense as te goal of the game it take out the objectives.

4. Damage Healed: This represents the damage healed by the Nasus player through other player heals or item heals. This could suggest that Nasus players are more successful with healing team compositions.

5. First Inhibitor Assist: The represents the first first inhibitor takedown assist. This would further suggest that Nasus Players who focus on objectives (especially the first inhibitor takedowns) win more games.

This model suggests that the top 5 features for unsuccessfull Nasus players are:

1. Trinity: This is the standard build item for Nasus, I am not sure why the model would be predicting that using this item leads to a losing game. Perhaps more data is needed or there needs to be a reorganization of the features.

2. Deaths: Deaths occur when the Nasus player is eliminated by a memeber of the enemy team. I would have expected this to be the deciding factor as deaths grant the enemy teams more gold which puts the Nasus at a disadvantage.

3. Total Damage Taken: This represents the total damage taken by this player. I would expect this column to be related to deaths. This could mean that taking lots of damage all the time could set back a Nasus player.

4. Spirit Visage: This is a defense item that grants magic resist that is a pretty standard item to build on Nasus. I am not sure why the model would be predicting that using this item leads to a losing game. Perhaps more data is needed or there needs to be a reorganization of the features.

5. Thornmail: This is a defense item that grants armor that is situational item to build on Nasus. Perhaps most in the dataset, a lot of Nasus players are building this item in the wrong situations.


Although this linear regression seems great on both accuracies and the model is easily explainable, I do not want to proceed with model as I want the final model to contain non-linear boundary conditions. 

### K Nearest Neighbors Classifier

In [18]:
# Import CLassifier
from sklearn.neighbors import KNeighborsClassifier

In [19]:
# Do a train test split
split = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)

# Scale the Features using the standard scaler
# Instantiate
std_scaler = StandardScaler()

# Fit and Transform
X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.fit_transform(X_test)

In [20]:
# Instantiate a default model
KNN = KNeighborsClassifier()

# fit to scaled training data
KNN_model = KNN.fit(X_train_scaled, y_train)
# score on scaled training data
print(f"The accuracy on the training set is: {KNN_model.score(X_train_scaled,y_train)}")

# score on scaled test data 
print(f"The accuracy on the test set is: {KNN_model.score(X_test_scaled,y_test)}")

The accuracy on the training set is: 0.7983922149354771
The accuracy on the test set is: 0.6835871404399323


This is a not a great score. This is probably due to the vast amounts of categorical data that we have encoded into the dataframe. Due to this fact, it probably isn't a good idea use the KNN classifier because we have probably imposed a notion of inter-category distance. Another thing to note is that there are large class imbalances within the features of the dataset, we would need to account for this using by implementing weighed voting as a hyperparameter. As such we will not proceed any further with the KNN classifier.

### Decision Trees

In [17]:
# import 
from sklearn.tree import DecisionTreeClassifier

In [21]:
# Do a train test split
split = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)

# Scale the Features using the standard scaler (NOT NEEDED FOR DTs)
# Instantiate
#std_scaler = StandardScaler()

# Fit and Transform
#X_train_scaled = std_scaler.fit_transform(X_train)
#X_test_scaled = std_scaler.fit_transform(X_test)

In [23]:
# Instantiate a default model
DT = DecisionTreeClassifier()

# fit to scaled training data
DT_model = DT.fit(X_train, y_train)
# score on training data
print(f"The accuracy on the training set is: {DT_model.score(X_train,y_train)}")

# score on test data 
print(f"The accuracy on the test set is: {DT_model.score(X_test,y_test)}")

The accuracy on the training set is: 1.0
The accuracy on the test set is: 0.800761421319797


Clearly the decision tree has overfit to the training data as expected. The accuracy on the test set is at ~80% which is not bad. Perhaps we can reduce the overfitting by modifying hyperparams, and increase the accuracy on the test set by using the random forest classifier. The Random Forest Classifier is the proccess of bagging many decision tree classifiers. Bagging composed of two processes:

1. Bootstrapping - Taking small subsets of the data trainning decision tree models on the independent subsamples.

2. Aggregating - Combining all of the models predictions and taking the average.

Just to note, an advantage of using decision trees and random forest classifiers is that I do not need to scale the data. This may save me some time when developing the functions for app.

### Random Forest Classifier

In [24]:
# import
from sklearn.ensemble import RandomForestClassifier

In [25]:
# Do a train test split
split = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)
# Scaling is not needed for any decision tree classifiers

In [26]:
# Instantiate
random_forest = RandomForestClassifier()

# Fit
random_forest_model = random_forest.fit(X_train, y_train)

# Score on train data
print(f"The accuracy on the training set is: {random_forest_model.score(X_train,y_train)}")

# score on test data 
print(f"The accuracy on the test set is: {random_forest_model.score(X_test,y_test)}")

The accuracy on the training set is: 1.0
The accuracy on the test set is: 0.8688663282571912


Just using the default random forest has already increase the accuracy on the test set by ~6.8%. This is quite the increase, althought it is still not as good as logistic regression which sits at ~89% on the test set. I could one other non-linear claissifier, the Support Vector Machines with kernel tricks, and then try to bag the models, however due to time constraints I will select the RandomForestClassifier as the model that I want to proceed with for developing my App.